In [1]:
from pathlib import Path

In [2]:
import datetime

In [3]:
import pandas as pd

In [4]:
import requests

In [5]:
import os

## Configuraciones de Variables

Definimos una ruta de directorio base

In [6]:
BASE_FILE_DIR = Path("/media/juniordavid/JUAN/Documents/python/DS/farmacias/copypharm/")


creamos un string que luego podemos editar a gusto con el método `.format()`

In [7]:
file_path_crib = 'data/{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv'

Creamos una lista e diccionarios que contengan diccionarios con claves `name` y `url` de cada una de las dataset a extraer

In [8]:
categories = [
    {
        "name": "farmacias",
        "url": "http://datos.salud.gob.ar/dataset/39117f8f-e2bc-4571-a572-15a6ce7ea9e1/resource/19338ea7-a492-4af3-b212-18f8f4af9184/download/establecimientos-farmacias-enero-2021.csv"
    }
]

## Farmacias

In [9]:
farmacias = categories[0]
farmacias

{'name': 'farmacias',
 'url': 'http://datos.salud.gob.ar/dataset/39117f8f-e2bc-4571-a572-15a6ce7ea9e1/resource/19338ea7-a492-4af3-b212-18f8f4af9184/download/establecimientos-farmacias-enero-2021.csv'}

In [10]:
r = requests.get(farmacias["url"])

In [11]:
r

<Response [200]>

In [12]:
r.encoding = 'utf-8'

In [13]:
now = datetime.datetime.now()
now

datetime.datetime(2022, 3, 24, 21, 15, 16, 791624)

In [14]:
file_path = file_path_crib.format(category=farmacias["name"], year=now.year, month=now.month, day=now.day)
file_path

'farmacias/2022-03/farmacias-24-03-2022.csv'

In [15]:
farmacias_path = BASE_FILE_DIR / file_path
farmacias_path

PosixPath('/media/juniordavid/JUAN/Documents/python/DS/farmacias/copypharm/farmacias/2022-03/farmacias-24-03-2022.csv')

In [16]:
farmacias_path.parent.mkdir(parents=True, exist_ok=True)

In [17]:
with open(farmacias_path, "w") as f:
    f.write(r.text)

## Transform

In [18]:
df = pd.read_csv(farmacias_path)

In [19]:
df.head()

,establecimiento_id,establecimiento_nombre,localidad_id,localidad_nombre,provincia_id,provincia_nombre,departamento_id,departamento_nombre,cod_loc,cod_ent,origen_financiamiento,tipologia_id,tipologia_nombre,tipologia,cp,domicilio,sitio_web
0,70260072329721,PUJOL,26007020000,PUERTO MADRYN,26,CHUBUT,7,BIEDMA,20,0.0,Privado,70,FARMACIA,Farmacia ambulatoria comercial,9120,LOS MAITENES 1538,NaN
1,70100352324743,SALUDFARMA,10035020000,BELEN,10,CATAMARCA,35,BELÉN,20,0.0,Privado,70,FARMACIA,Farmacia ambulatoria comercial,4750,Mitre 14,NaN
2,70064412318286,MARSICO,6441030015,LA PLATA,6,BUENOS AIRES,441,LA PLATA,30,15.0,Privado,70,FARMACIA,Farmacia ambulatoria comercial,1900,Avenida 7 445,NaN
3,70340492347884,PERALTA HNOS,34049010000,CLORINDA,34,FORMOSA,49,PILCOMAYO,10,0.0,Privado,70,FARMACIA,Farmacia ambulatoria comercial,3610,Avenida San Martin 470,NaN
4,70140142334991,VALENTINO,14014010000,CORDOBA,14,CÓRDOBA,14,CAPITAL,10,0.0,Privado,70,FARMACIA,Farmacia ambulatoria comercial,5000,Catamarca 1047,NaN


In [20]:
#df.columns
df.columns

Index(['establecimiento_id', 'establecimiento_nombre', 'localidad_id',
       'localidad_nombre', 'provincia_id', 'provincia_nombre',
       'departamento_id', 'departamento_nombre', 'cod_loc', 'cod_ent ',
       'origen_financiamiento', 'tipologia_id', 'tipologia_nombre',
       'tipologia', 'cp', 'domicilio', 'sitio_web'],
      dtype='object')

Renombrar las columnas

In [21]:
renamed_cols = {
    'establecimiento_id': 'id',
    'establecimiento_nombre': 'nombre',
    'localidad_id': 'id_localidad',
    'localidad_nombre': 'localidad',
    'provincia_id': 'id_provincia',
    'provincia_nombre': 'provincia',
    'departamento_id': 'id_departamento',
    'departamento_nombre': 'nombre_departamento',
    'cod_loc': 'cod_localidad',
    'tipologia_id': 'id_tipologia',
    'tipologia_nombre': 'nombre_tipologia',
    'cp': 'codigo_postal',
    'sitio_web': 'web',
}

In [22]:
df = df.rename(columns = renamed_cols)

In [33]:
df.columns
df.set_index("id")

,nombre,id_localidad,localidad,id_provincia,provincia,id_departamento,nombre_departamento,codigo_postal,domicilio,web
id,,,,,,,,,,
70260072329721,PUJOL,26007020000,PUERTO MADRYN,26,CHUBUT,7,BIEDMA,9120,LOS MAITENES 1538,NaN
70100352324743,SALUDFARMA,10035020000,BELEN,10,CATAMARCA,35,BELÉN,4750,Mitre 14,NaN
70064412318286,MARSICO,6441030015,LA PLATA,6,BUENOS AIRES,441,LA PLATA,1900,Avenida 7 445,NaN
70340492347884,PERALTA HNOS,34049010000,CLORINDA,34,FORMOSA,49,PILCOMAYO,3610,Avenida San Martin 470,NaN
70140142334991,VALENTINO,14014010000,CORDOBA,14,CÓRDOBA,14,CAPITAL,5000,Catamarca 1047,NaN
...,...,...,...,...,...,...,...,...,...,...
70460212355713,FARMACOST,46021020000,ANILLACO,46,LA RIOJA,21,CASTRO BARROS,5301,Coronel Nicolas Barros s/n Barrio Virgen del F...,NaN
70421472354613,FARMACIA MUSSO,42147030000,TRENEL,42,LA PAMPA,147,TRENEL,6369,9 DE JULIO 545,NaN
70940142195567,FARMACIA- HOSPITAL REGIONAL USHUAIA,94014020000,USHUAIA,94,TIERRA DEL FUEGO,14,USHUAIA,9410,12 de octubre 65,NaN


In [24]:
cols = [
    'id',
    'nombre',
    'id_localidad',
    'localidad',
    'id_provincia',
    'provincia',
    'id_departamento',
    'nombre_departamento',
    'codigo_postal',
    'domicilio',
    'web',
]

In [25]:
df = df[cols]

In [26]:
df

,id,nombre,id_localidad,localidad,id_provincia,provincia,id_departamento,nombre_departamento,codigo_postal,domicilio,web
0,70260072329721,PUJOL,26007020000,PUERTO MADRYN,26,CHUBUT,7,BIEDMA,9120,LOS MAITENES 1538,NaN
1,70100352324743,SALUDFARMA,10035020000,BELEN,10,CATAMARCA,35,BELÉN,4750,Mitre 14,NaN
2,70064412318286,MARSICO,6441030015,LA PLATA,6,BUENOS AIRES,441,LA PLATA,1900,Avenida 7 445,NaN
3,70340492347884,PERALTA HNOS,34049010000,CLORINDA,34,FORMOSA,49,PILCOMAYO,3610,Avenida San Martin 470,NaN
4,70140142334991,VALENTINO,14014010000,CORDOBA,14,CÓRDOBA,14,CAPITAL,5000,Catamarca 1047,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13672,70460212355713,FARMACOST,46021020000,ANILLACO,46,LA RIOJA,21,CASTRO BARROS,5301,Coronel Nicolas Barros s/n Barrio Virgen del F...,NaN
13673,70421472354613,FARMACIA MUSSO,42147030000,TRENEL,42,LA PAMPA,147,TRENEL,6369,9 DE JULIO 545,NaN
13674,70940142195567,FARMACIA- HOSPITAL REGIONAL USHUAIA,94014020000,USHUAIA,94,TIERRA DEL FUEGO,14,USHUAIA,9410,12 de octubre 65,NaN
13675,70420702154608,FARMACIA INSTITUCIONAL E.A. MANUEL FREIRE,42070030000,GUATRACHE,42,LA PAMPA,70,GUATRACHÉ,6309,Primera Junta Nº 158,NaN


In [27]:
df.dtypes

id                      int64
nombre                 object
id_localidad            int64
localidad              object
id_provincia            int64
provincia              object
id_departamento         int64
nombre_departamento    object
codigo_postal           int64
domicilio              object
web                    object
dtype: object

In [34]:
df_cordoba=df[df["provincia"]=='CÓRDOBA'][['id','nombre','id_localidad', 'id_departamento','codigo_postal', 'domicilio']]
df_cordoba.set_index('id')

,nombre,id_localidad,id_departamento,codigo_postal,domicilio
id,,,,,
70140142334991,VALENTINO,14014010000,14,5000,Catamarca 1047
70140142333750,HP FARMA V,14014010000,14,5000,Elías Yofre 1050
70140422333755,FARMAVIDA IX,14042170000,42,5900,Sarmiento 1966
70140212333758,GRAMIGNA,14021200000,21,5107,Avenida Martín Tissera 2997
70140142333759,GENERAL PAZ NUEVOCENTRO,14014010000,14,5000,Avenida Duarte Quirós 1400
...,...,...,...,...,...
70140142335135,FARMACIA YORIO SEBASTIAN,14014010000,14,5000,Octavio Pinto 3113
70140142335138,FARMACIA FARMANEXUS II,14014010000,14,5000,Avenida General Paz 298
70140142335147,FARMACIA JACOBO,14014010000,14,5000,Tristán Malbrán 4224


In [29]:
df_cordoba.dtypes

id                  int64
nombre             object
id_localidad        int64
id_departamento     int64
codigo_postal       int64
domicilio          object
dtype: object

In [30]:
df.groupby('localidad', as_index=False).count()[
    ['id_localidad','localidad']
]

,id_localidad,localidad
0,1,1 DE MAYO
1,1,17 DE AGOSTO
2,2,1A. SECCION
3,28,25 DE MAYO
4,4,28 DE NOVIEMBRE
...,...,...
1404,2,YUTO
1405,9,ZAPALA
1406,45,ZARATE
1407,1,ZELAYA


In [39]:
df_localidad=df[df['provincia']=="CÓRDOBA"].groupby(["id_localidad", 'localidad'], as_index=False).count()[["id_localidad", 'localidad']].set_index('id_localidad')
df_localidad

,localidad
id_localidad,
14007210,SANTA ROSA DE CALAMUCHITA (SANTA ROSA DE CALAM...
14021250,SALSIPUEDES (SALSIPUEDES)
14035120,VILLA HUIDOBRO (EST. CAÑADA VERDE)
14084120,GENERAL LAVALLE
14147330,VILLA LA BOLSA
...,...
14182080000,CANALS
14182130000,IDIAZABAL
14182150000,LABORDE


In [41]:
df_departamento=df[df['provincia']=="CÓRDOBA"].groupby(['id_departamento', 'nombre_departamento'],as_index=False).count()[['id_departamento', 'nombre_departamento']].set_index('id_departamento')

df_departamento

,nombre_departamento
id_departamento,
7,CALAMUCHITA
14,CAPITAL
21,COLÓN
28,CRUZ DEL EJE
35,GENERAL ROCA
42,GENERAL SAN MARTÍN
49,ISCHILÍN
56,JUÁREZ CELMAN
63,MARCOS JUÁREZ
